In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/test/rmgpy/test_data'}

In [3]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = ['Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 
                               'DFT_QCI_thermo',
                           'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
'SulfurLibrary', 'SulfurGlarborgH2S','SABIC_aromatics'],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'F_Abstraction',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [4]:
family = database.kinetics.families["F_Abstraction"]


In [5]:
family.clean_tree()

In [6]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ERROR:root:234.0
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early


167.74422240257263


In [7]:
len(family.groups.entries)

362

In [8]:
family.groups.entries

{'Root': <Entry index=0 label="Root">,
 'Root_1R->O': <Entry index=1 label="Root_1R->O">,
 'Root_N-1R->O': <Entry index=2 label="Root_N-1R->O">,
 'Root_1R->O_3R->O': <Entry index=3 label="Root_1R->O_3R->O">,
 'Root_1R->O_N-3R->O': <Entry index=4 label="Root_1R->O_N-3R->O">,
 'Root_N-1R->O_3R->O': <Entry index=5 label="Root_N-1R->O_3R->O">,
 'Root_N-1R->O_N-3R->O': <Entry index=6 label="Root_N-1R->O_N-3R->O">,
 'Root_1R->O_3R->O_Ext-1O-R': <Entry index=7 label="Root_1R->O_3R->O_Ext-1O-R">,
 'Root_1R->O_3R->O_Ext-3O-R_4R!H->C': <Entry index=8 label="Root_1R->O_3R->O_Ext-3O-R_4R!H->C">,
 'Root_1R->O_3R->O_Ext-3O-R_N-4R!H->C': <Entry index=9 label="Root_1R->O_3R->O_Ext-3O-R_N-4R!H->C">,
 'Root_1R->O_N-3R->O_1O-u0': <Entry index=10 label="Root_1R->O_N-3R->O_1O-u0">,
 'Root_1R->O_N-3R->O_N-1O-u0': <Entry index=11 label="Root_1R->O_N-3R->O_N-1O-u0">,
 'Root_N-1R->O_3R->O_Ext-3O-R': <Entry index=12 label="Root_N-1R->O_3R->O_Ext-3O-R">,
 'Root_N-1R->O_3R->O_1BrCClFHINPSSi-u0': <Entry index=13 l

In [9]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.1637709140777588


In [10]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

1.0895483493804932


In [11]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

0.997779130935669


In [12]:
len(templateRxnMap)

362

In [13]:
family.clean_tree_rules()

In [14]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3430: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


30.43414616584778


In [15]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.16359996795654297


In [16]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

1.4269556999206543


In [17]:
errors.values()

dict_values([-1.8466575103969072, -2.2602340040157864, 0.9985206428523006, 1.0011656240893232, 0.17692000370842784, -1.232710123854251, 0.1544278371209444, 2.256366488740556, 1.9331576542641438, -0.033540246916862235, -2.605347020088544, 0.9568321931631067, 1.0722451695455988, 0.06285407958815796, 2.1621836289092893, 0.06696409054233748, 3.2817837915739734, -0.4315456945420855, -0.28973687707665763, 0.611210279203468, -1.2122218321841745, -1.2586716241358014, -0.43730486628587095, 0.9347841709320327, 1.3157825147034319, -0.16400358229455192, -1.789894677966687, 1.3294508688518412, 1.9117195755472645, -6.590800112756416, 1.2381584425096714, -0.04047629113759992, 0.938364607309824, 3.0276454343964705, 0.93763310420488, 0.3698643327979772, 4.409064906888481, -2.685312907646222, -0.9447524451706435, 7.820092034771557, 0.9855159992836674, -4.959902082912337, -0.045748508987798024, 0.6988158403893256, -0.9178673157212317, 1.9736549714977514, 0.9753538413776409, 1.170986845373704, -1.03062045

In [18]:
plt.figure(figsize=(10,8))
plt.hist(np.abs(list(errors.values())),bins=30, density=True)
plt.title('Decision Tree Estimator',fontsize=18)
#plt.ylim(0,.4)
plt.xlim(0,15)
plt.ylabel('Probability density',fontsize=18)
plt.xlabel(r'$|Ln(k_{est}/k_{rxn})|$',fontsize=18)

Text(0.5, 0, '$|Ln(k_{est}/k_{rxn})|$')

In [ ]:
#family.save('/Users/mattjohnson/RMGCODE/RMG-database/input/kinetics/families/')

In [19]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/home/khalil.nor/Code/RMG-database/input/kinetics/families/F_Abstraction


In [20]:
family.save(save_path)